In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Replace with your actual file path
file_path = '/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/Health_and_Personal_Care.jsonl'

# Load the .jsonl file into a pandas DataFrame
df = pd.read_json(file_path, lines=True)

# Display the first 5 rows of the DataFrame
print("DataFrame Loaded Successfully!")
df.head()


DataFrame Loaded Successfully!


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,[],B07TDSJZMR,B07TDSJZMR,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-02-06 00:49:35.902,3,True
1,5,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,[],B08637FWWF,B08637FWWF,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2020-11-02 22:03:06.880,3,True
2,5,Fantastic,I have been suffering a couple months with hee...,[],B07KJVGNN5,B07KJVGNN5,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,2019-07-24 11:13:58.905,0,True
3,4,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",[],B007HY7GC2,B092RP73CX,AEZGPLOYTSAPR3DHZKKXEFPAXUAA,2022-09-04 02:29:02.725,7,True
4,1,Not for me,Didn't do a thing for me. Not saying they don'...,[],B08KYJLF5T,B08KYJLF5T,AEQAYV7RXZEBXMQIQPL6KCT2CFWQ,2022-01-20 23:53:07.262,0,True


In [ ]:
# number of reviews
print(len(df))

print(list(df.columns))

494121
['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase']


In [ ]:
# count of all ratings to check if there's imbalance
df['rating'].value_counts()

,count
rating,
5,301713
1,69564
4,57000
3,36949
2,28895


There seems to be a class imbalance. 5 star reviews seem to dominate a lot.

In [ ]:
df.isnull().sum()

,0
rating,0
title,0
text,0
images,0
asin,0
parent_asin,0
user_id,0
timestamp,0
helpful_vote,0
verified_purchase,0


There doesn't seem to be any null values.

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier # Import Random Forest
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Preprocessing

In [ ]:
df.dropna(subset=['text', 'rating'], inplace=True)
print(f"\nDataFrame shape after dropping rows with missing text/rating: {df.shape}")

# Combine 'title' and 'text'
# Fill missing titles with an empty string BEFORE concatenation
df['title'] = df['title'].fillna('')
df['review_full'] = df['title'] + ' ' + df['text']

# Text Cleaning Setup
nltk.download('punkt_tab', quiet=True) # Download the specific resource needed
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # 1. Lowercasing
    text = text.lower()
    # 2. Remove HTML tags (if any)
    text = re.sub(r'<.*?>', '', text)
    # 3. Remove punctuation and special characters (keeping only letters and whitespace)
    text = re.sub(r'[^a-z\s]', '', text)
    # 4. Tokenization
    tokens = word_tokenize(text)
    # 5. Remove Stop Words and Lemmatize
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 1] # Keep words with length > 1
    # 6. Join back into string
    return ' '.join(cleaned_tokens)

print("\nStarting text cleaning (this may take a while)...")
# Apply cleaning function - Monitor progress if needed for large datasets
# Consider df['review_full'].parallel_apply(clean_text) using libraries like pandarallel if speed is critical
df['review_cleaned'] = df['review_full'].apply(clean_text)
print("Text cleaning completed.")

# Drop rows where cleaning might have resulted in empty strings
df.dropna(subset=['review_cleaned'], inplace=True)
df = df[df['review_cleaned'].str.strip() != ''] # Ensure no empty strings after cleaning
print(f"DataFrame shape after cleaning and removing empty reviews: {df.shape}")

NameError: name 'df' is not defined

In [ ]:
df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,review_full,review_cleaned
0,4,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,[],B07TDSJZMR,B07TDSJZMR,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-02-06 00:49:35.902,3,True,12 mg is 12 on the periodic table people! Mg f...,mg periodic table people mg magnesium review c...
1,5,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,[],B08637FWWF,B08637FWWF,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2020-11-02 22:03:06.880,3,True,Save the lanet using less plastic. Love these ...,save lanet using less plastic love easy multit...
2,5,Fantastic,I have been suffering a couple months with hee...,[],B07KJVGNN5,B07KJVGNN5,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,2019-07-24 11:13:58.905,0,True,Fantastic I have been suffering a couple month...,fantastic suffering couple month heel pain pla...
3,4,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",[],B007HY7GC2,B092RP73CX,AEZGPLOYTSAPR3DHZKKXEFPAXUAA,2022-09-04 02:29:02.725,7,True,It holds the water and makes bubbles. That's ...,hold water make bubble thats bought cheap want...
4,1,Not for me,Didn't do a thing for me. Not saying they don'...,[],B08KYJLF5T,B08KYJLF5T,AEQAYV7RXZEBXMQIQPL6KCT2CFWQ,2022-01-20 23:53:07.262,0,True,Not for me Didn't do a thing for me. Not sayin...,didnt thing saying dont


Saving the processed data frame in a file (pickle, parquet and csv) as a checkpoints.

In [ ]:
df.to_pickle('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.pkl')

In [ ]:
df.to_parquet('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.parquet')

In [ ]:
df.to_csv('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.csv', index=False)

#Loading from existing parquet file

In [ ]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.parquet')

# --- 3. Feature Engineering (TF-IDF) ---

In [ ]:
# Define Features (X) and Target (y)
X = df['review_cleaned']
y = df['rating']

# Initialize TF-IDF Vectorizer
# You can experiment with parameters like max_features, ngram_range etc. later
# max_features=5000 limits the vocabulary size, can speed up training and reduce memory
# tfidf_vectorizer = TfidfVectorizer(max_features=10000, min_df=5, max_df=0.7) # Example parameters
tfidf_vectorizer = TfidfVectorizer() # Example parameters
print("\nInitializing TF-IDF Vectorizer...")




Initializing TF-IDF Vectorizer...


# --- 4. Data Splitting ---

In [ ]:
print("Splitting data into Training and Testing sets (80/20)...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 20% for testing
    random_state=42,  # for reproducibility
    stratify=y        # IMPORTANT: keep class distribution same in train/test due to imbalance
)
print(f"Training set size: {len(X_train)} samples")
print(f"Testing set size: {len(X_test)} samples")


Splitting data into Training and Testing sets (80/20)...
Training set size: 395056 samples
Testing set size: 98764 samples


# --- 5. Apply TF-IDF & Model Training ---

In [ ]:
import time

# Fit TF-IDF on Training data and Transform Training data
print("\nApplying TF-IDF to training data...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
print(f"TF-IDF applied to training data. Shape: {X_train_tfidf.shape}")

# Transform Test data using the *same* fitted vectorizer
print("Applying TF-IDF to testing data...")
X_test_tfidf = tfidf_vectorizer.transform(X_test)
print(f"TF-IDF applied to testing data. Shape: {X_test_tfidf.shape}")

# --- Model Training (Random Forest) ---
print("\nTraining Random Forest model...")
print("Parameters: n_estimators=200, criterion='gini', max_depth=None, class_weight='balanced', n_jobs=-1")
# Note: Training Random Forest can take significantly longer than Logistic Regression
rf_model = RandomForestClassifier(
    n_estimators=200,       # Number of trees in the forest
    criterion='gini',       # Function to measure the quality of a split
    max_depth=None,         # Expand nodes until leaves are pure (no limit)
    random_state=42,        # For reproducibility
    n_jobs=-1,              # Use all available CPU cores for training
    class_weight='balanced' # Adjust weights inversely proportional to class frequencies
)
start_time = time.time()
rf_model.fit(X_train_tfidf, y_train)
print("Model training completed.")
end_time = time.time()
training_duration = end_time - start_time

minutes = int(training_duration // 60)
seconds = training_duration % 60
print(f"Total training time: {minutes} minutes and {seconds:.2f} seconds.")

# --- >>> ADD THIS SECTION TO SAVE THE MODEL AND VECTORIZER <<< ---
import joblib
import os

# Define paths within your Google Drive to save the files
# Make sure the directory exists or create it
save_dir = '/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/saved_models' # CHANGE path if needed
os.makedirs(save_dir, exist_ok=True) # Create directory if it doesn't exist

model_save_path = os.path.join(save_dir, 'rf_rating_model.joblib')
vectorizer_save_path = os.path.join(save_dir, 'tfidf_vectorizer.joblib')

print(f"\nSaving trained model to: {model_save_path}")
joblib.dump(rf_model, model_save_path)
print("Model saved successfully.")

print(f"Saving TF-IDF vectorizer to: {vectorizer_save_path}")
joblib.dump(tfidf_vectorizer, vectorizer_save_path)
print("Vectorizer saved successfully.")
# --- >>> END OF SAVING SECTION <<< ---


Applying TF-IDF to training data...
TF-IDF applied to training data. Shape: (395056, 149434)
Applying TF-IDF to testing data...
TF-IDF applied to testing data. Shape: (98764, 149434)

Training Random Forest model...
Parameters: n_estimators=200, criterion='gini', max_depth=None, class_weight='balanced', n_jobs=-1
Model training completed.
Total training time: 345 minutes and 40.66 seconds.

Saving trained model to: /content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/saved_models/rf_rating_model.joblib
Model saved successfully.
Saving TF-IDF vectorizer to: /content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/saved_models/tfidf_vectorizer.joblib
Vectorizer saved successfully.


# --- 6. Model Evaluation ---

In [ ]:
# --- Model Evaluation ---
print("\nEvaluating model performance on the Test set...")

# Predict on the test set
y_pred = rf_model.predict(X_test_tfidf)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nOverall Accuracy: {accuracy:.4f}")

# Detailed Classification Report
print("\nClassification Report:")
# Use zero_division=0 to handle cases where a class might have no predicted samples
print(classification_report(y_test, y_pred, target_names=[str(i) for i in sorted(y.unique())], zero_division=0))

# Confusion Matrix
print("\nConfusion Matrix:")
# Ensure labels match the unique sorted values in y for correct indexing
labels = sorted(y.unique())
print(pd.DataFrame(confusion_matrix(y_test, y_pred, labels=labels),
                   index=[f'Actual_{i}' for i in labels],
                   columns=[f'Predicted_{i}' for i in labels]))


Evaluating model performance on the Test set...

Overall Accuracy: 0.7355

Classification Report:
              precision    recall  f1-score   support

           1       0.75      0.66      0.70     13909
           2       0.90      0.13      0.22      5779
           3       0.75      0.17      0.27      7387
           4       0.85      0.17      0.28     11395
           5       0.73      0.99      0.84     60294

    accuracy                           0.74     98764
   macro avg       0.80      0.42      0.46     98764
weighted avg       0.76      0.74      0.68     98764


Confusion Matrix:
          Predicted_1  Predicted_2  Predicted_3  Predicted_4  Predicted_5
Actual_1         9239           45           62           10         4553
Actual_2         1648          732          119           14         3266
Actual_3          874           27         1220           48         5218
Actual_4          213            2          119         1893         9168
Actual_5          370  

# --- EVALUATION FROM CHECKPOINT ---

In [ ]:
import joblib
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from google.colab import drive # Make sure drive is mounted if running in a new session

# --- Mount Drive (if needed) ---
drive.mount('/content/drive')

# --- Define Paths (MUST match the paths used for saving) ---
save_dir = '/content/drive/MyDrive/DADM/saved_models' # CHANGE path if needed
model_save_path = os.path.join(save_dir, 'rf_rating_model.joblib')
vectorizer_save_path = os.path.join(save_dir, 'tfidf_vectorizer.joblib')

# --- Load the Model and Vectorizer ---
print(f"Loading trained model from: {model_save_path}")
loaded_rf_model = joblib.load(model_save_path)
print("Model loaded successfully.")

print(f"Loading TF-IDF vectorizer from: {vectorizer_save_path}")
loaded_tfidf_vectorizer = joblib.load(vectorizer_save_path)
print("Vectorizer loaded successfully.")

# --- Ensure you have your Test Data ---
# If X_test_tfidf and y_test are not in memory, you need to recreate them.
# IMPORTANT: Use the *loaded_tfidf_vectorizer* to transform X_test
# Example (if you need to recreate X_test_tfidf):
# print("Reloading and preparing test data...")
# Load original df or X_test, y_test if saved separately
# file_path = '/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/Health_and_Personal_Care.jsonl'
# df = pd.read_json(file_path, lines=True)
# # ... (repeat necessary preprocessing steps on df: dropna, combine text, clean text) ...
# X = df['review_cleaned']
# y = df['rating']
# _, X_test_text, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # Regenerate text split
# print("Applying loaded TF-IDF to testing data...")
# X_test_tfidf = loaded_tfidf_vectorizer.transform(X_test_text) # Use LOADED vectorizer
# print(f"TF-IDF applied to testing data. Shape: {X_test_tfidf.shape}")

# --- Evaluate the Loaded Model ---
# Assuming X_test_tfidf and y_test are available in your current session
print("\nEvaluating loaded model performance on the Test set...")

# Predict on the test set using the loaded model
y_pred_loaded = loaded_rf_model.predict(X_test_tfidf) # Use loaded model

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred_loaded)
print(f"\nOverall Accuracy: {accuracy:.4f}")

# Detailed Classification Report
print("\nClassification Report:")
# Ensure labels match the unique sorted values in y_test or original y for correct indexing
labels = sorted(y_test.unique()) # Use y_test.unique() to be safe
print(classification_report(y_test, y_pred_loaded, target_names=[str(i) for i in labels], zero_division=0))

# Confusion Matrix
print("\nConfusion Matrix:")
print(pd.DataFrame(confusion_matrix(y_test, y_pred_loaded, labels=labels),
                   index=[f'Actual_{i}' for i in labels],
                   columns=[f'Predicted_{i}' for i in labels]))